In [160]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(URL(
        account='to57169',
        user='yang',
        password='yang123!',
        warehouse = 'fivetran_warehouse',
        database='fivetran',
        schema='bbbnw'

    )
)

In [162]:
#features_to_use = ["BUSINESS","STATE"， "ZIP”]
#Read businessname, state, and zipcode from master and 3rd party dataset, excluding all empty ZIP codes. 
master_bus_st = pd.read_sql_query('SELECT DISTINCT business, state, left(zip, 5) as zip FROM master WHERE zip IS NOT NULL', engine)
business_df = pd.read_sql_query('SELECT businessname as business, mailingstatecode as state, left(mailingzip, 5) as zip FROM businesslicense WHERE mailingzip IS NOT NULL', engine)
active_business_df = pd.read_sql_query('SELECT business_name as business, state as state, left(zip, 5) as zip FROM activeorbusinesses WHERE zip IS NOT NULL', engine)
prof_license_df = pd.read_sql_query('SELECT owners as business, state as state, left(zip, 5) as zip FROM professionallicense WHERE zip IS NOT NULL', engine)
#right_join_master_business = pd.read_sql_query('SELECT businesslicense.businessname as business, businesslicense.mailingstatecode as state, businesslicense.mailingzip as zip FROM businesslicense RIGHT JOIN master ON master.business = businesslicense.businessname', engine)

In [163]:
print(business_df)
print(active_business_df)
print(prof_license_df)


                                                business state    zip
0                                  A & A ROOFING CO, INC    AK  99707
1                          A AND W WHOLESALE COMPANY INC    AK  99707
2                     ELAINE S. BAKER & ASSOCIATES, INC.    AK  99501
3                           DON ABEL BUILDING SUPPLY INC    AK  99801
4                                 ALASKA AIRLINES, INC      WA  98168
5                                 ALASKA FLOR-WALL, INC.    AK  99501
6                                  FAIRBANKS GOLDPANNERS    AK  99707
7                                 ALASKA INDIAN ARTS INC    AK  99827
8                        ALASKA INDUSTRIAL HARDWARE, INC    AK  99501
9                                       ALASKA LANDS INC    AK  99603
10                                       AVIS RENT A CAR    AK  99519
11                          ALASKA SAUSAGE COMPANY, INC.    AK  99509
12              ALASKA VILLAGE ELECTRIC COOPERATIVE, INC    AK  99503
13            COOK I

In [164]:
#Inner joins on just the business names would not be helpful as the same business could be in different zip-codes
#as such these records should be handled manually 
#removing duplicates from the database and doing a right join on all 3 fields instead. 
#No records are reduced by the right join

business_df.drop_duplicates(['business','zip'], inplace = True)
prof_license_df.drop_duplicates(['business','zip'], inplace = True)
active_business_df.drop_duplicates(['business','zip'], inplace = True)

right_join_master_bus = pd.merge(master_bus_st, business_df, how = 'right', on=['business','zip','state'])
right_join_master_prof = pd.merge(master_bus_st, prof_license_df, how = 'right', on=['business','zip','state'])
right_join_master_active = pd.merge(master_bus_st, active_business_df, how = 'right', on=['business','zip','state'])

#these right joins can now be used for further processing

print(right_join_master_bus)
print(right_join_master_prof)
print(right_join_master_active)

                                                business state    zip
0                                 ALASKAN POND AT JUNEAU    AK  99801
1                                      THE KNIFE GRINDER    AK  99567
2                            BIG GAME BACKCOUNTRY GUIDES    OR  97522
3                                       DOUGLAS ELECTRIC    AK  99824
4                          100TH MONKEY YOGA STUDIO, LLC    AK  99615
5                      WELLSPRING HOME HEALTH CENTER LLC    AK  99654
6                                  THE POWER OF LAUGHTER    AK  99504
7                                     TIGHTEN YOUR FANCY    AK  99507
8                           APEX BUILDING SOLUTIONS, LLC    AK  99654
9                                    BROOKS BUILDERS LLC    AK  99603
10                                                  KYUK    AK  99559
11               CRIGHTON/COOPER CONSTRUCTION AND DESIGN    AK  99501
12                                           DESIGN TILE    AK  99645
13                  

In [71]:
#processing of inner joins is not required
'''def pre_process_dataframes(df,inner_join):
    inner_join.dropna(subset=['zip'], inplace = True) 
    inner_join[['zip']] = inner_join[['zip']].astype(object)
    inner_join.drop_duplicates('business', inplace=True)
    df1 = pd.concat([df, inner_join])
    df1.drop_duplicates(keep=False,inplace=True)
    return df1
print(inner_join_master_professional)
print(inner_join_master_active)'''

                                               business state    zip
0                               Nationstar Mortgage LLC    TX  75261
2                                        Art Church Inc    WA  98119
3                                 The Covollo Group LLC    WA  98072
4                             Pioneer Fire & Safety LLC    WA  98507
5                            Stew's Heating Service LLC    AK  99709
6                                 Epic Construction LLC    WA  98005
7                                      Melissa Williams    WA  99201
9                Kerr McVey Sheaffer & Associates, P.C.    AK  99503
10                                     Team Cutters Inc    AK  99709
11                                     Vigor Alaska LLC    AK  99901
12                               Electrical Systems Inc    WA  98146
13                          Dream Home Construction LLC    OR  97504
14                      Top of the World Mechanical LLC    AK  99645
15                            Gran

In [165]:
from fuzzywuzzy import fuzz

#To be implemented. We will find how to combine similarity ratio of business name and zip code.
#Currently, we only consider similarity of business name.
def computeSimilarity(business_temp, master_temp):
    for index,row in business_temp.iterrows():
        name_similarity = 0
        zip_similarity = 0
        max_combine = 0
        master_name = "xxx"
        master_zip = 0
        business_name = row['business']
        business_zip = row['zip']

        for index1, row1 in master_temp.iterrows():
            business = row1['business']
            zipcode = row1['zip']
            name_similar = fuzz.ratio(business_name.lower(), business.lower())
            zip_similar = fuzz.ratio(business_zip, zipcode)

            if(zip_similar<40):
                combine_similar = name_similar
            else:
                combine_similar = name_similar * 0.8 + zip_similar * 0.2

            if(combine_similar > max_combine):
                name_similarity = name_similar
                zip_similarity = zip_similar
                max_combine = combine_similar
                master_name = business
                master_zip = zipcode
        business_temp.loc[index, 'name_match_percent'] = name_similarity
        business_temp.loc[index, 'zip_match_percent'] = zip_similarity
        business_temp.loc[index, 'combined_percent'] = max_combine
        business_temp.loc[index, 'master_businessname'] = master_name
        business_temp.loc[index, 'master_zip'] = master_zip

In [166]:
def markDuplicate(business_temp):
    for index, row in business_temp.iterrows():
        if(row['combined_percent']>85):
            business_temp.loc[index, 'duplicate?'] = 1
        else:
            business_temp.loc[index, 'duplicate?'] = 0

In [167]:

right_join_master_bus['zip_match_percent'] = pd.Series(0, index=right_join_master_bus.index)
right_join_master_bus['name_match_percent'] = pd.Series(0, index=right_join_master_bus.index)
right_join_master_bus['combined_percent'] = pd.Series(0, index=right_join_master_bus.index)
right_join_master_bus['master_businessname'] = pd.Series("", index=right_join_master_bus.index)
right_join_master_bus['master_zip'] = pd.Series(0, index=right_join_master_bus.index)
right_join_master_bus['duplicat?'] = pd.Series(0, index=right_join_master_bus.index)  

print(right_join_master_bus)


                                                business state    zip  \
0                                 ALASKAN POND AT JUNEAU    AK  99801   
1                                      THE KNIFE GRINDER    AK  99567   
2                            BIG GAME BACKCOUNTRY GUIDES    OR  97522   
3                                       DOUGLAS ELECTRIC    AK  99824   
4                          100TH MONKEY YOGA STUDIO, LLC    AK  99615   
5                      WELLSPRING HOME HEALTH CENTER LLC    AK  99654   
6                                  THE POWER OF LAUGHTER    AK  99504   
7                                     TIGHTEN YOUR FANCY    AK  99507   
8                           APEX BUILDING SOLUTIONS, LLC    AK  99654   
9                                    BROOKS BUILDERS LLC    AK  99603   
10                                                  KYUK    AK  99559   
11               CRIGHTON/COOPER CONSTRUCTION AND DESIGN    AK  99501   
12                                           DESIGN

In [168]:
#for state in list_of_states: 


list_of_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
                 "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
                 "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""] 

#for state in list_of_states:
master_temp = master_bus_st[(master_bus_st.state=='CO')]
business_temp = right_join_master_bus[(right_join_master_bus.state=='CO')] 
computeSimilarity(business_temp, master_temp)
markDuplicate(business_temp) 

business_temp.to_csv("bus_temp_CO.csv")


c:\program files (x86)\python36-32\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
c:\program files (x86)\python36-32\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
